<a href="https://colab.research.google.com/github/Seyisha/Seyisha/blob/main/kowope_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 71 kB/s 


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cat_
import seaborn as sns
import lightgbm as lgb

In [17]:
train=pd.read_csv("/content/Train_exp.csv")

In [18]:
test=pd.read_csv("/content/Test_exp.csv")

In [19]:
class Func():
    def __init__(self,train,label, test, model, model_type,random_type):
        self.train,self.label,self.test=train,label,test
        self.model,self.model_type=model,model_type
        self.random_type=random_type
      
        assert  model_type in ("catboost","xgboost","lightgbm"), "incorrect model_type"

    def __call__(self, plot=True):
        return self.fit(plot)

    def fit(self,plot):
        def catboost_fit(X_train,X_test,y_train,y_test):
            self.model.fit(X_train,y_train,eval_set=[(X_test,y_test)], early_stopping_round =500,
                           verbose=50,use_best_model=True)
            x_test_predict=self.model.predict.proba(X_test)[:,1]
            x_train_predict=self.model.predict.proba(X_train)[:,1]
            self.val_p[test_index]=x_test_predict
            self.test_p += self.model.predict.proba(self.test)[:,1]
            return x_test_predict,x_train_predict


        def xgboost_fit(X_train,X_test,y_train,y_test):
            self.model.fit(X_train,y_train, eval_set=[(X_test,y_test)], early_stopping_round=30,
                           verbose= True, eval_metric="auc")
            x_test_predict=self.model.predict.proba(X_test,ntree_limit=self.model.get_booster().best_ntree_limit)[:,1]
            x_train_predict=self.model.predict.proba(X_train,ntree_limit = self.model.get_booster().best_ntree_limit)[:,1]
            self.val_p[test_index]=x_test_predict
            self.test_p += self.model.predict.proba(self.test, num_iteration = self.model.best_iteration_)[:,1]
            return x_test_predict,x_train_predict


        def lightgbm_fit(X_train,X_test,y_train,y_test):
            self.model.fit(X_train,y_train, eval_set=[(X_test,y_test)],verbose=True,
                           eval_metric ="auc",early_stopping_round=30 )
            x_test_predict=self.model.predict.proba(X_test, num_iteration = self.model.best_iteration_)[:,1]
            x_train_predict=self.model.predict.proba(X_train,num_iteration=self.model.best_iteration_)[:,1]
            self.val_p[test_index]=x_test_predict
            self.test_p +=self.model.predit.proba(self.test, num_iteration = self.model.best_iteration_)[:,1]
            return x_test_predict, x_train_predict

        self.val_p=np.zeros(self.train.shape[0])
        mean_val=[]
        mean_train=[]
        self.test_p=n.zeros(self.test.shape[0])
        split=5
        kf=StratifiedKFold(n_splits=split)
        for fold_count, (train_index, test_index) in enumerate(kf.split(self.train, self.label)):
            X_train,X_test=self.train.iloc[train_index],self.train.iloc[test_index]
            y_train,y_test=self.label.iloc[train_index],self.label.iloc[test_index]

            
            print(f"================================Fold{fold_count+1}====================================")
            if self.model_type=='catboost': x_test_predict,x_train_predict=catboost_fit(X_train,X_test,y_train,y_test)
            elif self.model_type=='xgboost':x_test_predict,x_train_predict=xgboost_fit(X_train,X_test,y_train,y_test)
            elif self.model_type=='lightgbm':x_test_predict,x_train_predict=lightgbm_fit(X_train,X_test,y_train,y_test)

            print('\nValidation scores', roc_auc_score(y_test, x_test_predict), log_loss(y_test, x_test_predict))
            print('Training scores', roc_auc_score(y_train, x_train_predict), log_loss(y_train, x_train_predict))
            mean_val.append(roc_auc_score(y_test, x_test_predict))
            mean_train.append(roc_auc_score(y_train, x_train_predict))

        if plot:
            feat_imp= pd.DataFrame(sorted(zip(self.model.feature_importance_,self.tain.columns)), columns =['value','Features'])
            plt.figure(figsize=(30,25))
            sns.barplot(x='value', y='Features', data=feat_imp.sort_values(by='values', ascending =False))
            plt.ylabel("Feature Importance score ")
            plt.show()
        print(np.mean(mean_val), np.mean(mean_train), np.std(mean_val))
        return self.val_p, self.test_p/split, self.model




        

        





In [27]:
train.columns

Index(['user_id', 'REGION', 'TENURE', 'MONTANT', 'FREQUENCE_RECH', 'REVENUE',
       'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
       'ZONE1', 'ZONE2', 'MRG', 'REGULARITY', 'TOP_PACK', 'FREQ_TOP_PACK',
       'CHURN'],
      dtype='object')

In [28]:
print(train.MRG.value_counts(dropna= False), 'train length:', len(train) , '\n')
print(test.MRG.value_counts(dropna= False),'test length:', len(test))

NO    400000
Name: MRG, dtype: int64 train length: 400000 

NO    100000
Name: MRG, dtype: int64 test length: 100000


In [29]:
train.drop(["user_id","MRG",],1, inplace =True)
test.drop(["user_id","MRG",],1, inplace =True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [37]:
ntrain=train.shape[0]
ntest=test.shape[0]
data=pd.concat((train,test)).reset_index(drop= True)


In [45]:
data['REVENUE-MONTANT'] = data['REVENUE'] - data['MONTANT']
data['REVENUE/MONTANT'] = data['REVENUE'] / data['MONTANT']

In [46]:
data['TENURE']=data['TENURE'].map({'k<24':24,'I 18-21 month': 18, 'H 15-18 month': 15, 'G 12-15 month':12,
                                             'J 21-24 month': 21, 'F 9-12': 9, 'E 6-9 month':6, 'D 3-6 month':3})

In [47]:
#return Avg of the tenure/month 
data['TENURE_AVG']=data['TENURE'].map({'k<24':(24+27)/2,'I 18-21 month': (18+21)/2, 'H 15-18 month': (15+18)/2, 'G 12-15 month':(12+15)/2,
                                             'J 21-24 month': (24+21)/2, 'F 9-12': (12+9)/2, 'E 6-9 month':(6 +9)/2, 'D 3-6 month':(3 +6)/2})

In [50]:
data['TENURE/FREQUENCE_RECH'] = data['TENURE_AVG'] / data['FREQUENCE_RECH']
data['TENURE/REGULARITY'] = data['TENURE_AVG'] / data['REGULARITY']

In [54]:
region=data.groupby('REGION').mean()
region. drop('CHURN', 1, inplace= True )
cols = []
for i in region.columns:
    if i != 'REGION':
        region[i+'_reg_mean_all'] = region[i]
        region.drop(i, 1, inplace = True)
        cols.append(i+'_reg_mean_all')

data = pd.merge(data, region, on='REGION', how = 'left')
for col in cols: data[col+'_freq'] = data[col].map(data[col].value_counts().to_dict())/len(data)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


In [55]:
data['freq*montant'] = data['FREQUENCE'] * data['MONTANT']
data['freq*rech'] = data['FREQUENCE'] * data['FREQUENCE_RECH']
data['freq*revenue'] = data['FREQUENCE'] * data['REVENUE']
data['freq*segment'] = data['FREQUENCE'] * data['ARPU_SEGMENT']

data['freq/montant'] =  data['MONTANT']/ data['FREQUENCE']
data['freq/rech'] = data['FREQUENCE'] / data['FREQUENCE_RECH']
data['freq/revenue'] = data['FREQUENCE'] / data['REVENUE']
data['freq/segment'] = data['FREQUENCE'] / data['ARPU_SEGMENT']

In [56]:
data['data/reg'] = data['DATA_VOLUME'] / data['REGULARITY']
data['net/reg'] = data['ON_NET'] / data['REGULARITY']
data['montant-rech/freq'] = (data['MONTANT'] - data['FREQUENCE_RECH']) / data['FREQUENCE']
data['segment/reg'] = data['ARPU_SEGMENT'] / data['REGULARITY']

In [58]:
data['reg_log'] = np.log1p(data['REGULARITY'])
data['rech_log'] = np.log1p(data['FREQUENCE_RECH'])
data['data_log'] = np.log1p(data['DATA_VOLUME'])
data['montant_log'] = np.log1p(data['MONTANT'])
data['rev_log'] = np.log1p(data['REVENUE'])
data['freq_log'] = np.log1p(data['FREQUENCE'])

In [59]:
data['reglog-montlog'] = data['reg_log'] - data['montant_log']
data['revlog/montlog'] = data['REVENUE'] / data['montant_log']
data['tenure/rechlog'] = data['TENURE_AVG'] / data['rech_log']
data['reglog-datalog'] = data['reg_log'] - data['data_log']

In [60]:
drop = ['REGION', 'TOP_PACK']
data.drop(drop, 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
